# Test the TCF module interactively

In [1]:
from temporal_collaborative import TCF

temporal_collaborative_filtering = TCF("tgbl-review", decay=0.5, k=10, factor=50)

raw file found, skipping download
Dataset directory is  /Users/shahrad/projs/TGB_baseline/tgb_env/lib/python3.11/site-packages/tgb/data/tgbl_review
loading processed file


In [2]:
temporal_collaborative_filtering.train()

Training the temporal collaborative filtering model for O(352637^2) edges.


Training:   0%|          | 0/17070 [00:00<?, ?it/s]

Training: 100%|██████████| 17070/17070 [00:11<00:00, 1459.01it/s]


In [3]:
# output max and min of values in the bank dict
print("Max value in bank: ", max(temporal_collaborative_filtering.bank.values()))
print("Min value in bank: ", min(temporal_collaborative_filtering.bank.values()))


Max value in bank:  3162741256.8294806
Min value in bank:  4.440892098500626e-16


In [4]:
# print the similarities 
print("Similarity matrix: ", temporal_collaborative_filtering.similarities.shape)

Similarity matrix:  (352637, 352637)


In [5]:
# temporal_collaborative_filtering._get_most_similar_to(339189, is_source=False)

In [6]:
# a = torch.Tensor([i for i in range(10)])
# temporal_collaborative_filtering.predict(a,a,a)

# temporal_collaborative_filtering.predict(torch.Tensor([180293]), torch.Tensor([339189]), torch.Tensor([12300]))

# Validation using the test function

In [7]:
# temporal_collaborative_filtering._get_most_similar_to(343942)

In [8]:
# [(k,v) for (src, k), v in temporal_collaborative_filtering.bank.items() if src == 343942]

In [9]:
# [(k,v) for (src, k), v in temporal_collaborative_filtering.bank.items() if src == 8521]

In [10]:
temporal_collaborative_filtering.val_test(split_mode="val")

Validating the temporal collaborative filtering model.
